In [1]:
import os
os.getpid()

18425

In [2]:
import random
list2 = []
file_data = open('amazon_review_less_than_300_chars_balanced.csv')
for row in file_data:
    list2.append(row)## Data pre-processing module
    
list3 = []
for i in range(len(list2)):
    list3.append((list2[i][-2],list2[i][:-3]))
    
random.seed(10)
random.shuffle(list3)

lenth = len(list3)
train_list = list3[0:int(lenth*0.8)]
test_list = list3[int(lenth*0.8):]




In [4]:

import torch
from torch.utils.data.sampler import BatchSampler, Sampler, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertModel, AdamW, BertConfig
from tqdm import tqdm

class Corpora():
    """
        A class that holds the texts and class labels.
    """
    
    # Modification: l represent train_list or test_list
    
    def __init__(self, l):
        """
            file_path: on MAGIC, it is either
                        data-badassnlp/cola_public/raw/in_domain_train.tsv or
                        data-badassnlp/cola_public/raw/in_domain_dev.tsv
                    on Google Colab, change them to the corresponding paths.
        """

        self.sentences = [s[1] for s in tqdm(l)]
        self.labels = [int(s[0]) for s in tqdm(l)]
        
        
class TextClassificationDataSet(Dataset):
    """
        Define a dataset consisting of pairs of (sequence_of_word_indices, class_label).
        class_label is either 0 or 1.
    """
    def __init__(self, corpora, tokenizer: BertTokenizer):
        """
            This function will tokenize the sentences in self.corpora,
            and pad the word indices properly.
            The tokenization and padding should be done using BERT's API.

                See https://huggingface.co/docs/transformers/internal/tokenization_utils
                    for the API of tokenizer
                See https://huggingface.co/docs/transformers/preprocessing
                    for examples of using the API
                Also see the BERT paper
                    "BERT: Pre-training of Deep Bidirectional Transformers forLanguage Understanding"
                regarding how paddings are done.

            We turn the entire corpora into a big tensor in a batch, rather than
                doing that in the __getitem__ function, as batch processing can speed up tokenization
        
            Then self.input_ids, self.attention_mask and self.labels
            will be created as tensors of shape
                torch.Size([num_sentences, max_padded_sentence_length])
                torch.Size([num_sentences, max_padded_sentence_length])
                torch.Size([num_sentences])

            corpora: an object of the Corpora class representing some raw classified texts.
            tokenizer: must be the BERTTokenizer loaded from the BERT/download folder
                        in order to have the same ids for the words in any vocabulary.
                        See the loading codes below.
        """
        self.corpora = corpora
        self.tokenizer = tokenizer

        ### Your codes go here (30 points) ###

        # Step 1 (10 points): figure out the max original sentence length with the tokenizer's padding option.
        #           Please read the documentation of tokenizer to figure out the arguments and the return value.
       
        # --------------------------------------------------- 
            # Answer: 
            # By tokenizer.pad
            # model_max_len=1000000000000000019884624838656
        # --------------------------------------------------- 
        
        # Step 2 (10 points): find max_length = maximum length of all training/dev sentences after padding with [CLS] and [SEP]
        #           use the max_length to inform the tokenizer and create padded input.
        #           Please read the documentation of tokenizer to figure out the arguments and the return value.

        # --------------------------------------------------- 

            
            # dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
            # "Token_type_ids" seems to be all 0s
            # "Attention_mask" seems to be all 1s (why)
            # The "input_ids" seems to give a distinct value for different words. (which is nice)
        d =  tokenizer.batch_encode_plus(corpora.sentences)
        
            # Here, we did not try to find maxium length. (If it will cause any trouble)
        
        padded_encoded_inputs = tokenizer.pad(d)
        
        # --------------------------------------------------- 
        


        # Step 3 (10 points): extract the necessary tensors padded_encoded_inputs.input_ids
        #           and padded_encoded_inputs.attention_mask
        #           and make the self.input_ids and self.attention_mask tensors.
        #           also make the self.labels in a tensor.
        self.input_ids = torch.tensor(padded_encoded_inputs.input_ids)
        self.attention_mask = torch.tensor(padded_encoded_inputs.attention_mask)
        self.labels = torch.tensor(corpora.labels)
        
    def __len__(self):
        return len(self.corpora.sentences)

    def __getitem__(self, idx):
        """
            Return the idx-th of the rows the self.input_ids, self.attention_mask, self.labels in this order.
            Don't do BERT tokenization here as that will be slow.
        """
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]

In [5]:
from torch import nn

class BERTClassifier(nn.Module):
    
    def __init__(self, BERT_model, hidden_layer_size, num_classes):
        
        super(BERTClassifier, self).__init__()

        # loaded pretrained model
        self.bert = BERT_model
        
        # simple neural network that convert embedding of the first token to a class
        self.classifier = nn.Sequential(
            nn.Linear(BERT_hidden_size, hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, num_classes)
        )
        
    def forward(self, input_ids, attention_mask):
        """
        The following two arguments are tensors from a mini-batch of the input_ids
            and attention_mask returned by the BERT tokenizer.
            
            input_ids: a tensor of shape [batch_size, max_length]
            attention_mask: a tensor of shape [batch_size, max_length]
            
            return: the logits of the sentences in the batch tensor of shape [batch_size, 1, 2]
        """
        # see https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        # and https://huggingface.co/docs/transformers/main_classes/output
        # for the return value of the forward function of BERT

        ### Your codes go here (30 points) ###
        z = self.bert.forward(input_ids, attention_mask)
        return torch.softmax(self.classifier(z[1]).unsqueeze(1), dim=-1) 
        
        return logits

In [6]:
# make sure you check the availability of GPU when you set the device ID.
device = torch.device(2)

BATCH_SIZE = 8
num_classes = 5
classifier_hidden_size = 128

## if using MAGIC
BERT_PATH = '/data/badassnlp/bert-base-uncased/'
## if using Google Colab, you need to load the bert model after it downloads the model files.
## 

BERT_hidden_size = 768

N_EPOCHS = 5
CLIP = 1

In [7]:
## 'uncased' means all words are lowered-cased before tokenization
## 'base' means the smaller version of BERT (12 layers, 16 heads)
## un-comment one of the following two options.

# if using MAGIC, load from local BERT folder
tokenizer = BertTokenizer.from_pretrained(BERT_PATH, local_files_only=True)
BERT_model = BertModel.from_pretrained(BERT_PATH, num_labels = 2, output_attentions = False, output_hidden_states = False
).to(device)

## if using Colab, load from automatically downloaded files. Downloading can take half a minute
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# BERT_model = BertModel.from_pretrained("bert-base-cased", num_labels = 2, output_attentions = False, output_hidden_states = False).to(device)


## if using MAGIC
print("creating training corpora ... ")
training_corpora = Corpora(train_list)
print("creating training dataset ... ")
training_dataset = TextClassificationDataSet(training_corpora, tokenizer)
print("creating training iterator ... ")
training_iterator = DataLoader(training_dataset, sampler = RandomSampler(training_dataset), batch_size=BATCH_SIZE)

## if using MAGIC
print("creating test corpora ... ")
dev_corpora = Corpora(test_list)
print("creating test dataset ... ")
dev_dataset = TextClassificationDataSet(dev_corpora, tokenizer)
print("creating test iterator ... ")
dev_iterator = DataLoader(dev_dataset, sampler = SequentialSampler(dev_dataset), batch_size=BATCH_SIZE)

classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)

Some weights of the model checkpoint at /data/badassnlp/bert-base-uncased/ were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


creating training corpora ... 


100%|██████████| 400000/400000 [00:00<00:00, 2193292.22it/s]


creating training dataset ... 
creating training iterator ... 
creating test corpora ... 


100%|██████████| 100000/100000 [00:00<00:00, 2035704.27it/s]

creating test dataset ... 


creating test iterator ... 


In [8]:
list(dev_iterator)[0][0].shape

torch.Size([8, 128])

In [9]:
from torch import optim
optimizer = optim.Adam(classifier.parameters())
criterion = nn.CrossEntropyLoss(reduction = 'sum')

num_epochs_train = 0

from tqdm import tqdm

In [35]:
def train(model, iterator, optimizer, criterion, clip):
    """
        model: an BERTClassifier object
        iterator: a DataLoader object
        optimizer: torch optimizer
        criterion: the crossentropyloss
        clip: to be used with torch.nn.utils.clip_grad_norm_

        return: average loss over the training instances (sentences) in the DataLoader.
    """
    model.train()
    
    epoch_loss = 0
    num_batchs = 0
    total_instances = 0
    global num_epochs_train
    num_epochs_train += 1
    for i, batch in tqdm(enumerate(iterator)):
        num_batchs += 1
        optimizer.zero_grad() 
        tmp = optimizer.state_dict()
        tmp["param_groups"][0]["lr"] = 0.00002/(num_epochs_train)
        optimizer.load_state_dict(tmp)

        ### Your codes go here (10 points) ###

        # Step 1 (5 points): get the tensors from this mini-batch and increase the total_instances
        # make sure tensors are moved to GPU.
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]-1).to(device)
        
        # Step 2 (5 points): call the forward function of the model and find the output logits
        # then calculate the loss. Then cumulate the epoch_loss
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) # if squeeze here, why unsqueeze previous
        loss.backward()

        # Clips gradient norm of an iterable of parameters.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
        total_instances += BATCH_SIZE

    return epoch_loss / total_instances

confusion_matrix = []
num_epochs = 0

def evaluate(model, iterator, criterion):
    """
        Find the loss (criterion) of the model over instances in this DataLoader (iterator)
        Same logic as the above train function, except no gradient calculation
        and no update of the parameter. 
        
        return: average loss over the training instances (sentences) in the DataLoader.
    """
    model.eval()

    epoch_loss = 0
    total_instances = 0
    
    confusion_matrix.append(torch.zeros(5,5))
    global num_epochs
    
    for i, batch in tqdm(enumerate(iterator)):
        
        ### Your codes go here (5 points) ###
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]-1).to(device)
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) # if squeeze here, why unsqueeze previous
        
        epoch_loss += loss.item()
        total_instances += BATCH_SIZE
        

        for i in range(len(y)):
            row = y[i]
            col = torch.argmax((logits.squeeze(1))[i])
            confusion_matrix[num_epochs][row][col] += 1
    num_epochs += 1

    return epoch_loss / total_instances

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [36]:
import time

best_test_loss = float('inf')
training_losses = []
test_losses = []

for epoch in range(N_EPOCHS):  
    print("epoch start: ", epoch)  
    start_time = time.time()
    training_loss = train(classifier, training_iterator, optimizer, criterion, CLIP)
    training_losses.append(training_loss)
    test_loss = evaluate(classifier, dev_iterator, criterion)
    test_losses.append(test_loss)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if test_loss < best_test_loss:
        best_test_loss = test_loss 
        torch.save(classifier.state_dict(), 'best_model.pt')
        

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s', end='')
    print(f'\tTrain Loss: {training_loss:.3f} | Test Loss: {test_loss:.3f}')

epoch start:  0


50000it [2:12:03,  6.31it/s]
12500it [05:04, 41.06it/s]


Epoch: 01 | Time: 137m 8s	Train Loss: 1.199 | Test Loss: 1.278
epoch start:  1


50000it [2:12:17,  6.30it/s]
12500it [05:04, 41.03it/s]


Epoch: 02 | Time: 137m 22s	Train Loss: 1.188 | Test Loss: 1.278
epoch start:  2


40658it [1:47:39,  6.33it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

4202it [11:06,  6.37it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

19782it [52:17,  6.33it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0

Epoch: 04 | Time: 137m 18s	Train Loss: 1.172 | Test Loss: 1.277
epoch start:  4


50000it [2:13:57,  6.22it/s]
12500it [05:04, 41.03it/s]

Epoch: 05 | Time: 139m 1s	Train Loss: 1.165 | Test Loss: 1.277


In [45]:
print(confusion_matrix[9])
print(test_losses)

tensor([[13747.,  4101.,  1436.,   349.,   331.],
        [ 5560.,  9472.,  3783.,   565.,   498.],
        [ 1802.,  4282., 10263.,  3023.,   541.],
        [  219.,   377.,  2468., 12145.,  4968.],
        [  137.,   225.,   361.,  2384., 16963.]])
[1.3146484128952027, 1.2859456231069566, 1.279662517466545, 1.27645985683918, 1.2793248378181457, 1.2779897089242935, 1.2776662415266038, 1.2773806825733185, 1.2773248982572556, 1.2765346831417084]


In [44]:
distance = 0
diagnal = 0

ep = 10
for i in range(5):
    for j in range(5):
        distance+= confusion_matrix[ep-1][i][j]*abs(i-j)
        if i == j:
            diagnal += confusion_matrix[ep-1][i][j]
        
print("dist:" ,distance)
print("diag: ",diagnal)

dist: tensor(46478.)
diag:  tensor(62590.)


In [28]:
int(train_list[10][0])-1

print(list(dev_iterator)[0][2]-1)

print(classifier.forward(list(dev_iterator)[0][0].cuda(2),list(dev_iterator)[0][1].cuda(2)))

tensor([1, 2, 1, 1, 1, 2, 3, 3])
tensor([[[4.6859e-01, 3.9175e-01, 1.3949e-01, 1.3469e-04, 4.0148e-05]],

        [[1.6594e-07, 3.1954e-06, 9.9999e-01, 1.9403e-06, 5.7594e-09]],

        [[1.0000e+00, 3.1152e-07, 4.0230e-08, 1.0431e-07, 3.2509e-11]],

        [[2.6730e-01, 5.7599e-01, 1.5535e-01, 8.8893e-04, 4.7313e-04]],

        [[3.2141e-06, 9.9999e-01, 1.9507e-06, 9.3135e-10, 3.5361e-09]],

        [[8.0248e-07, 1.1253e-07, 2.6564e-06, 9.9999e-01, 3.1390e-06]],

        [[1.1651e-03, 1.4594e-02, 1.3898e-01, 6.9422e-01, 1.5104e-01]],

        [[1.0000e+00, 4.7848e-07, 3.6542e-08, 7.4078e-08, 3.2185e-11]]],
       device='cuda:2', grad_fn=<SoftmaxBackward0>)


In [33]:
test_list[0:58]
count = [0,0,0,0,0]
for i in test_list:
    count[int(i[0])-1]+=1
count

[19964, 19878, 19911, 20177, 20070]